In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd

### Load dataset

In [2]:
df = pd.read_csv('final_dataset.csv', low_memory=False)

In [3]:
df.head()

,REPORT_DATE,COUNTY_NAME,COUNTY_FIPS_NUMBER,PEOPLE_POSITIVE_CASES_COUNT,PROVINCE_STATE_NAME,CONTINENT_NAME,DATA_SOURCE_NAME,PEOPLE_DEATH_NEW_COUNT,COUNTRY_ALPHA_3_CODE,COUNTRY_SHORT_NAME,...,FERTILITY_RATE,PERCENT_POPULATION_AGED_10_24,PERCENT_POPULATION_AGED_15_64,PERCENT_POPULATION_AGED_65_OLDER,LIFE_EXPECTANCY,DAYS_AFTER_FIRST_DEATH,DAYS_AFTER_FIRST_CASE,DAYS_BETWEEN_FIRST_CASE_AND_FIRST_DEATH,PERCENT_DEATH_BY_POPULATION,PERCENT_CASES_BY_POPULATION
0,2020-02-17,Watonwan,27165.0,0,Minnesota,America,New York Times,0,USA,United States,...,1.9,20.0,65.0,16.0,80.0,NaN,27.0,39.0,NaN,NaN
1,2020-05-21,Watonwan,27165.0,42,Minnesota,America,New York Times,0,USA,United States,...,1.9,20.0,65.0,16.0,80.0,82.0,121.0,39.0,NaN,42000.0
2,2020-05-28,Watonwan,27165.0,64,Minnesota,America,New York Times,0,USA,United States,...,1.9,20.0,65.0,16.0,80.0,89.0,128.0,39.0,NaN,64000.0
3,2020-06-06,Watonwan,27165.0,80,Minnesota,America,New York Times,0,USA,United States,...,1.9,20.0,65.0,16.0,80.0,98.0,137.0,39.0,NaN,80000.0
4,2020-03-17,Watonwan,27165.0,0,Minnesota,America,New York Times,0,USA,United States,...,1.9,20.0,65.0,16.0,80.0,17.0,56.0,39.0,NaN,NaN


### Prepare dataset

In [4]:
df = df.drop_duplicates()

In [5]:
df_to_kmeans = df.drop(columns=[
                      'REPORT_DATE',
                      'PEOPLE_DEATH_NEW_COUNT', 
                      'PEOPLE_POSITIVE_NEW_CASES_COUNT', 
                      'DAYS_AFTER_FIRST_DEATH', 
                      'DAYS_AFTER_FIRST_CASE',
                      'COUNTY_NAME',
                      'COUNTY_FIPS_NUMBER',
                      'PEOPLE_POSITIVE_CASES_COUNT',
                      'PROVINCE_STATE_NAME',
                      'DATA_SOURCE_NAME',
                      'COUNTRY_ALPHA_3_CODE',
                      'PEOPLE_DEATH_COUNT',
                      'ISO',
                      'CONTINENT_NAME',
                      'PERCENT_DEATH_BY_POPULATION',
                      'PERCENT_CASES_BY_POPULATION'])

In [6]:
df_to_kmeans_without_country = df_to_kmeans.drop(columns=['COUNTRY_ALPHA_2_CODE', 'COUNTRY_SHORT_NAME'])

### Train kmeans

In [7]:
df_to_kmeans_without_country = df_to_kmeans_without_country.fillna(0)
df_to_kmeans_without_country = df_to_kmeans_without_country.replace('-', 0, regex=True)
df_to_kmeans_without_country = df_to_kmeans_without_country.replace(',', '', regex=True)

In [8]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_to_kmeans_without_country)

In [9]:
df_to_kmeans['CLUSTER']=kmeans.labels_

In [10]:
df_to_kmeans[df_to_kmeans['COUNTRY_ALPHA_2_CODE'] == 'BR'].head()

,COUNTRY_SHORT_NAME,COUNTRY_ALPHA_2_CODE,TOTAL_POPULATION,FERTILITY_RATE,PERCENT_POPULATION_AGED_10_24,PERCENT_POPULATION_AGED_15_64,PERCENT_POPULATION_AGED_65_OLDER,LIFE_EXPECTANCY,DAYS_BETWEEN_FIRST_CASE_AND_FIRST_DEATH,CLUSTER
378894,Brazil,BR,212.4,1.7,23.0,70.0,9.0,76.0,21.0,4
378921,Brazil,BR,212.4,1.7,23.0,70.0,9.0,76.0,21.0,4
378931,Brazil,BR,212.4,1.7,23.0,70.0,9.0,76.0,21.0,4
389551,Brazil,BR,212.4,1.7,23.0,70.0,9.0,76.0,21.0,4
389866,Brazil,BR,212.4,1.7,23.0,70.0,9.0,76.0,21.0,4


In [12]:
df_cluster = df_to_kmeans.loc[:, df_to_kmeans.columns.intersection([
                                         'COUNTRY_ALPHA_2_CODE', 
                                         'CLUSTER'])]

In [14]:
df_cluster = df_cluster.drop_duplicates()

In [15]:
df_cluster.head()

,COUNTRY_ALPHA_2_CODE,CLUSTER
0,US,0
30721,MP,3
205408,GU,1
225107,PR,1
294412,VI,1


In [16]:
df = pd.merge(df, df_cluster, how='left', left_on='COUNTRY_ALPHA_2_CODE', right_on='COUNTRY_ALPHA_2_CODE')

In [20]:
df.to_csv('dataset_to_tableau.csv')